In [27]:
%pip install e2b_code_interpreter

   ---------------------------------------- 0.0/102.8 kB ? eta -:--:--
   ---------------------------------------- 102.8/102.8 kB 5.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/137.6 kB ? eta -:--:--
   ---------------------------------------- 137.6/137.6 kB 8.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [47]:
from openai import OpenAI
from typing import List
import subprocess
import os
from e2b_code_interpreter import CodeInterpreter
import re
from concurrent.futures import ThreadPoolExecutor, as_completed

In [57]:
class LLM:
    def __init__(self, openai_api_key: str):
        self.client = OpenAI(api_key=openai_api_key)

    def create_completion(self, model, prompt, system_message) -> str:
        """
        Generate a completion using the OpenAI API.

        Parameters:
            - model (str): The model to use (e.g., "gpt-3.5").
            - messages (List[dict]): A list of messages for the chat completion.

        Returns:
            - str: The generated text.
        """
        try:
            response = self.client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": system_message},
                    {"role": "user",
                        "content": prompt}
                ]
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error in create_completion: {e}")
            return "An error occurred while processing this chunk."

In [58]:
class PlannerAgent:
    def __init__(self, llm: LLM):
        self.llm = llm

    def create_plan(self, query: str) -> str:
        """
        Generate a detailed plan for the given query using the LLM.

        Parameters:
            - query (str): The task or problem statement for which a plan is needed.

        Returns:
            - str: A detailed plan generated by the LLM.
        """
        # Define the system message and the prompt for the LLM
        system_message = (
            "You are a highly experienced project planning expert with advanced knowledge in machine learning and data science. "
            "Your task is to design a comprehensive and actionable plan for the given problem statement, ensuring that all critical aspects are addressed. "
            "Your plan should cover the full project lifecycle, from data acquisition and preprocessing to model development, training, and evaluation. "
            "The plan must include detailed methodologies, specific techniques, code snippets, and best practices. Provide a logical and structured sequence of steps to achieve the project's objectives."
        )

        prompt = (
            f"The following is a detailed task description that requires a thorough project plan:\n\n"
            f"Query: {query}\n\n"
            "Please generate a detailed, step-by-step plan to address this task. Your plan should include: "
            "1. **Understanding the Problem:** "
            "   - Analyze the problem statement and define the goals and objectives of the neural network. "
            "   - Determine the type of neural network most suitable for the task (e.g., feedforward, convolutional, recurrent, etc.). "
            "   - Identify the key requirements and constraints for the neural network. "
            "2. **Data Acquisition and Preprocessing:** "
            "   - Describe the process for acquiring and loading the data, including data sources, formats, and any APIs involved. "
            "   - Outline methods for data cleaning, handling missing values, outliers, and anomalies. "
            "   - Specify data transformation and normalization techniques that are appropriate for the type of neural network being developed. "
            "   - Detail file handling and formatting practices. "
            "3. **Model Development:** "
            "   - Provide a detailed design of the neural network architecture, including: "
            "     - Specifications of the network layers (e.g., types of layers, units, activation functions). "
            "     - The choice of algorithms and techniques for training the network. "
            "     - Optimization strategies, including the choice of optimization algorithms and tuning hyperparameters. "
            "     - Advanced techniques relevant to the neural network type (e.g., dropout, batch normalization). "
            "4. **Training Procedures:** "
            "   - Describe strategies for splitting the data into training, validation, and test sets, including the rationale behind these choices. "
            "   - Provide details on the training process, including: "
            "     - Training parameters such as epochs, batch sizes, learning rates, and regularization methods. "
            "     - Hardware and software requirements for efficient training. "
            "5. **Evaluation Strategies:** "
            "   - Outline methods for evaluating the neural network's performance, including: "
            "     - Metrics for assessment (e.g., accuracy, precision, recall, F1 score) and their relevance to the task. "
            "     - Techniques for visualizing and interpreting evaluation results (e.g., Precision-Recall curve, ROC curve). "
            "     - Guidance on using evaluation metrics to refine and improve the model. "
            "6. **Handling Special Considerations:** "
            "   - Address any specific challenges related to the neural network, such as: "
            "     - Managing imbalanced datasets. "
            "     - Incorporating domain-specific considerations or constraints. "
            "     - Ensuring the scalability and efficiency of the model. "
            "7. **Documentation and Best Practices:** "
            "   - Include documentation and best practices for implementation, including: "
            "     - Providing clear and structured code and explanations. "
            "     - Following best practices for neural network design and training. "
            "     - Ensuring reproducibility and maintainability of the model. "
            "Ensure that each step is detailed, actionable, and includes best practices and relevant resources. The plan should be clear, structured, and tailored to the specific neural network task described."
        )

        messages = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt}
        ]

        # Use the LLM to generate the plan
        plan = self.llm.create_completion("gpt-4", prompt, system_message)
        return plan

In [67]:
class CodeGeneratorAgent:
    def __init__(self, llm: LLM):
        self.llm = llm

    def generate_code(self, plan: str) -> str:
        """
        Generate code based on the detailed plan provided by the PlannerAgent.

        Parameters:
            - plan (str): The detailed plan for which code needs to be generated.

        Returns:
            - str: The generated code based on the plan.
        """
        system_message = (
            "You are a highly skilled code generation expert with extensive experience in translating detailed project plans into executable code. "
            "Your role is to convert the provided comprehensive plan into precise and functional code, ensuring it meets all requirements outlined in the plan. "
            "Your code should be well-structured, modular, and adhere to best coding practices. Include clear comments and explanations to facilitate understanding and maintenance."
        )

        prompt = (
            f"Based on the following detailed plan, generate executable Python code that fulfills all specified requirements:\n\n"
            f"Plan: {plan}\n\n"
            "Instructions for Code Generation: "
            "1. **Code Output Only:** "
            "   - Provide only the Python code necessary to fulfill the plan. Exclude any extra text, explanations, comments, or metadata. "
            "2. **Imports and Dependencies:** "
            "   - At the beginning of the code, include all relevant import statements for libraries and modules needed to execute the code as described in the plan. "
            "   - Ensure that all dependencies required by the plan are imported and available. "
            "3. **Code Organization:** "
            "   - Structure the code into well-defined functions, classes, or modules as needed. "
            "   - Maintain clear and logical organization to enhance readability and maintainability. "
            "4. **Implementation Details:** "
            "   - Implement the code according to the steps and requirements outlined in the plan. This includes data loading, preprocessing, model development, training, and evaluation. "
            "   - Ensure that the code correctly handles all specified tasks and follows the best practices described in the plan. "
            "5. **Error Handling and Validation:** "
            "   - Integrate error handling and validation checks where applicable to manage unexpected inputs or issues. "
            "6. **Best Practices:** "
            "   - Follow coding best practices, including efficient algorithms and clean code principles. "
            "   - Verify that the code is fully functional and adheres to the specifications given in the plan."
        )

        # Use the LLM to generate the code
        code = self.llm.create_completion(
            model="gpt-4", prompt=prompt, system_message=system_message)
        return self._clean_code(code)

    def regenerate_code(self, previous_code: str, test_results: str) -> str:
        """
        Regenerate code based on the previous code and test results.

        Parameters:
            - previous_code (str): The previously generated code that needs revision.
            - test_results (str): The results of tests performed on the previously generated code.

        Returns:
            - str: The revised code based on the test results.
        """
        system_message = (
            "You are a code revision expert with experience in improving code based on test results. Your task is to revise the given code to address the issues identified in the test results. "
            "The revised code should meet the specified requirements and fix any problems highlighted by the tests. Ensure that the revised code adheres to best practices and includes all necessary imports and dependencies."
        )

        prompt = (
            f"Based on the results provided below, please revise and improve the program to ensure it functions correctly. "
            "Generate a new program that meets the specified requirements and addresses any issues identified in the tests. Do not include the tests themselves in the output, only the revised code.\n\n"
            f"Previous Code:\n{previous_code}\n\n"
            f"Test Results:\n{test_results}\n\n"
            "Instructions for Code Improvement: "
            "1. **Code Output Only:** "
            "   - Provide only the Python code necessary to fulfill the revised plan. Exclude any extra text, explanations, comments, or metadata. "
            "2. **Imports and Dependencies:** "
            "   - Ensure that all relevant import statements for libraries and modules needed to execute the revised code are included. "
            "   - Verify that all dependencies required by the revised plan are imported and available. "
            "3. **Code Revisions:** "
            "   - Revise the code to fix any issues identified in the test results. Ensure that all problems are addressed and that the code is functional and reliable. "
            "   - Maintain clear and logical organization to enhance readability and maintainability. "
            "4. **Error Handling and Validation:** "
            "   - Update error handling and validation checks as needed to improve the code's robustness. "
            "5. **Best Practices:** "
            "   - Follow coding best practices, including efficient algorithms and clean code principles. "
            "   - Ensure that the revised code is fully functional and meets the requirements specified in the plan."
        )

        # Use the LLM to generate the revised code
        revised_code = self.llm.create_completion(
            model="gpt-4", prompt=prompt, system_message=system_message)
        return self._clean_code(revised_code)
    
    def _clean_code(self, code: str) -> str:
        """
        Clean up and format the generated code to ensure it's valid Python code.
        """
        # Strip any extraneous text or metadata from the code
        code = code.strip()
        # Add more cleaning logic if necessary
        return code

In [68]:
class TestGeneratorAgent:
    def __init__(self, llm: LLM):
        self.llm = llm

    def generate_tests(self, code: str, plan: str) -> str:
        """
        Generate a comprehensive test suite for the given code based on the provided plan.

        Parameters:
            - code (str): The generated code that needs to be tested.
            - plan (str): The detailed plan that describes the functionality and requirements of the code.

        Returns:
            - str: The generated test suite based on the code and plan.
        """
        system_message = (
            "You are an expert in creating test suites for code. Your role is to generate a comprehensive set of tests to ensure the correctness and functionality of the provided code. "
            "You should use the detailed plan and generated code to create meaningful test cases that cover all aspects of the code's functionality. Ensure that the tests are well-organized, effective, and follow best practices for testing. "
            "Make sure to include all necessary library imports in the test code to ensure it runs correctly. Also, create dummy data points directly in the code for testing purposes."
        )

        prompt = (
            f"Given the following code and the associated plan, generate a complete and relevant test suite to validate the functionality and correctness of the code:\n\n"
            f"Code:\n{code}\n\n"
            f"Plan:\n{plan}\n\n"
            "Instructions for Test Generation: "
            "1. **Test Coverage:** "
            "   - Identify and write tests for all major functionalities, edge cases, and requirements described in the plan. "
            "   - Ensure that the tests cover data preprocessing, model training, evaluation, and any other key components of the code. "
            "2. **Dummy Data Points:** "
            "   - Create dummy data points or sample inputs directly in the test code instead of reading from files. "
            "   - Ensure these data points are representative of the scenarios described in the plan. "
            "3. **Test Types:** "
            "   - Include unit tests to validate individual functions, methods, or classes in the code. "
            "   - Write integration tests to verify that different components of the code work together as expected. "
            "   - Add end-to-end tests if applicable, to ensure the overall system behaves correctly according to the plan. "
            "4. **Test Cases:** "
            "   - Create test cases with clear and descriptive names. "
            "   - Include test inputs, expected outputs, and any necessary setup or teardown procedures. "
            "   - Implement tests to cover both expected and edge cases, as well as error handling scenarios. "
            "5. **Testing Framework:** "
            "   - Use a suitable testing framework (e.g., pytest, unittest) based on the programming language and environment. "
            "   - Ensure that the tests are compatible with the code's structure and dependencies. "
            "6. **Documentation:** "
            "   - Add comments and documentation to the test code to explain the purpose and functionality of each test case. "
            "   - Ensure that the tests are easy to understand and maintain. "
            "7. **Imports:** "
            "   - Include all necessary imports required for the test suite to run successfully. "
            "   - Make sure that the imports are relevant and only include what is needed for the test cases. "
            "8. **Best Practices:** "
            "   - Follow best practices for writing tests, including effective use of assertions and ensuring test isolation. "
            "   - Verify that the test suite runs efficiently and provides clear feedback on the correctness of the code."
        )

        # Use the LLM to generate the test suite
        test_suite = self.llm.create_completion(
            model="gpt-4", prompt=prompt, system_message=system_message)
        return test_suite

    def regenerate_tests(self, code: str, test_results: str, plan: str) -> str:
        """
        Regenerate and improve the test suite based on previous test results.

        Parameters:
            - code (str): The generated code that needs to be tested.
            - test_results (str): The results of the previous test executions.
            - plan (str): The detailed plan that describes the functionality and requirements of the code.

        Returns:
            - str: The revised and improved test suite based on the previous test results.
        """
        system_message = (
            "You are an expert in improving test suites for code. Your role is to revise and enhance the test cases based on the provided test results to ensure comprehensive coverage and accuracy. "
            "Use the previous test results and the detailed plan to create improved and additional test cases where needed. Ensure the tests are well-organized, effective, and follow best practices."
        )

        prompt = (
            f"Given the following code, previous test results, and the associated plan, regenerate and improve the test suite to enhance its effectiveness:\n\n"
            f"Code:\n{code}\n\n"
            f"Previous Test Results:\n{test_results}\n\n"
            f"Plan:\n{plan}\n\n"
            "Instructions for Test Regeneration: "
            "1. **Review Test Results:** "
            "   - Analyze the previous test results to identify any gaps, failures, or areas for improvement. "
            "   - Determine which aspects of the code or plan were not adequately tested. "
            "2. **Enhance Test Coverage:** "
            "   - Revise existing test cases to address any identified gaps or issues. "
            "   - Add new test cases to cover any previously untested functionalities, edge cases, or scenarios. "
            "3. **Update Dummy Data Points:** "
            "   - Modify or add dummy data points directly in the test code if needed to improve test coverage or address specific issues found in the previous tests. "
            "4. **Test Types and Cases:** "
            "   - Ensure that unit tests, integration tests, and end-to-end tests are updated as necessary. "
            "   - Create or refine test cases with clear and descriptive names, including updated test inputs, expected outputs, and setup or teardown procedures. "
            "5. **Testing Framework:** "
            "   - Ensure that the updated test suite is compatible with the chosen testing framework (e.g., pytest, unittest) and the code's structure. "
            "6. **Documentation:** "
            "   - Update comments and documentation to reflect any changes or improvements in the test cases. "
            "   - Ensure the tests are easy to understand and maintain. "
            "7. **Imports:** "
            "   - Verify that all necessary imports are included in the updated test suite and that they are relevant. "
            "8. **Best Practices:** "
            "   - Follow best practices for test writing, including effective use of assertions and ensuring test isolation. "
            "   - Ensure that the revised test suite runs efficiently and provides clear feedback on code correctness."
        )

        # Use the LLM to regenerate the improved test suite
        improved_test_suite = self.llm.create_completion(
            model="gpt-4", prompt=prompt, system_message=system_message)
        return improved_test_suite

In [69]:
class CodeExecutorAgent:
    def __init__(self, api_key: str):
        self.api_key = api_key

    def execute_code(self, code: str) -> str:
        """
        Executes the provided code using the E2B test environment.
        
        :param code: The code snippet to execute.
        :return: The result of the execution, indicating success or any errors encountered.
        """
        try:
            # Initialize the E2B CodeInterpreter with the provided API key
            with CodeInterpreter(api_key=self.api_key) as sandbox:
                # Execute the code in the E2B environment
                execution = sandbox.notebook.exec_cell(code)
                # Check for standard output or errors
                result = "Passed"
        except Exception as e:
            result = f"Error during code execution: {e}"
        return result

    def execute_tests(self, code: str, tests: str) -> str:
        """
        Executes the provided code and test cases using the E2B test environment.
        
        :param code: The code snippet to test.
        :param tests: The test cases to execute.
        :return: The result of the test execution, indicating success or any errors encountered.
        """
        combined_code = f"{code}\n\n{tests}"
        try:
            # Initialize the E2B CodeInterpreter with the provided API key
            with CodeInterpreter(api_key=self.api_key) as sandbox:
                # Execute the combined code and tests in the E2B environment
                execution = sandbox.notebook.exec_cell(combined_code)
                # Check for standard output or errors
                result = "Passed"
        except Exception as e:
            result = f"Error during test execution: {e}"
        return result

    def _process_execution_result(self, execution) -> str:
        """
        Processes the result of the code or test execution.
        
        :param execution: The result object returned from the CodeInterpreter.
        :return: A string indicating the outcome of the execution.
        """
        # Assuming `execution` object has attributes like `output` and `error`
        if hasattr(execution, 'error'):
            if execution.error:
                return f"Error during execution: {execution.error}"
            else:
                return "Code executed successfully."
        elif hasattr(execution, 'output'):
            if 'Error' in execution.output:
                return f"Code execution failed: {execution.output}"
            return "Code executed successfully."
        else:
            return "Unknown execution result format."


In [70]:
def extract_python_code(text):
    """
    Extract the Python code from the provided text string.

    Args:
    text (str): The input text containing the Python code
    
    Returns:
    str: The extracted Python code
    """
    code_pattern = re.compile(r'```python(.*?)```', re.DOTALL)
    match = code_pattern.search(text)
    
    if match:
        return match.group(1).strip()
    else:
        return "No Python code found in the input text."

In [63]:
import concurrent.futures

In [72]:
def main():
    # Initialize agents
    OPEN_AI_APIKEY_PART1 = "sk-None-"
    OPEN_AI_APIKEY_PART2 = 'Rn5Gph5C4okKPr9PVD8AT3BlbkFJMBtKVLtpK3EjL7E1eSpq'
    OPEN_AI_APIKEY = OPEN_AI_APIKEY_PART1 + OPEN_AI_APIKEY_PART2
    llm = LLM(openai_api_key=OPEN_AI_APIKEY)
    planner = PlannerAgent(llm=llm)
    code_generator = CodeGeneratorAgent(llm=llm)
    test_generator = TestGeneratorAgent(llm=llm)
    code_executor = CodeExecutorAgent("e2b_8738475f82ec794a770b9b6e110aec14fe4d9589")
    
    query = "I have uploaded 20 files, with 5 files for each type of RNA sequence. For each RNA type, there are four files representing different experimental conditions and one control file. Build a recurrent neural network-based model to differentiate between experimental and control RNA sequences. For evaluation results, visualize the confusion matrix and precision-recall curves."

    # Generate detailed plan
    detailed_plan = planner.create_plan(query=query)

    # Initialize variables for results
    code_result = ""
    tests_result = ""
    count = 0
    max_iterations = 5

    while count < max_iterations:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            # Generate code and tests
            future_code = executor.submit(code_generator.generate_code, detailed_plan)
            generated_code = extract_python_code(future_code.result())
            
            future_tests = executor.submit(test_generator.generate_tests, generated_code, detailed_plan)
            generated_tests = extract_python_code(future_tests.result())

        # Save generated code and tests to files
        with open('generated_code.py', 'w') as code_file:
            code_file.write(generated_code)

        with open('test_generated_code.py', 'w') as test_file:
            test_file.write(generated_tests)

        with concurrent.futures.ThreadPoolExecutor() as executor:
            # Execute the code and tests
            future_code_result = executor.submit(code_executor.execute_code, generated_code)
            code_result = future_code_result.result()

            future_tests_result = executor.submit(code_executor.execute_tests, generated_code, generated_tests)
            tests_result = future_tests_result.result()

        if code_result == "Passed" and tests_result == "Passed":
            print("Code and tests passed successfully.")
            break

        # Regenerate code and tests if not passed
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future_code = executor.submit(code_generator.regenerate_code, generated_code, tests_result)
            generated_code = extract_python_code(future_code.result())
            
            future_tests = executor.submit(test_generator.regenerate_tests, generated_code, tests_result, detailed_plan)
            generated_tests = extract_python_code(future_tests.result())

        count += 1


    print("Code Result:")
    print(code_result + "\n")

    print("Tests Result:")
    print(tests_result + "\n")

    with open('final_output.txt', 'w') as file:
        # file.write("Generated Code:\n")
        # file.write(generated_code)
        # file.write("\n\nGenerated Tests:\n")
        # file.write(generated_tests)
        file.write("\n\nCode Result:\n")
        file.write(code_result)
        file.write("\n\nTests Result:\n")
        file.write(tests_result)

main()

Code and tests passed successfully.
Code Result:
Passed

Tests Result:
Passed



In [39]:
api_key = "your_openai_api_key_here"
OPEN_AI_APIKEY = 'sk-None-Rn5Gph5C4okKPr9PVD8AT3BlbkFJMBtKVLtpK3EjL7E1eSpq'
llm = LLM(openai_api_key=OPEN_AI_APIKEY)
planner = PlannerAgent(llm=llm)
code_generator = CodeGeneratorAgent(llm=llm)
test_generator = TestGeneratorAgent(llm=llm)
code_executor = CodeExecutorAgent("e2b_8738475f82ec794a770b9b6e110aec14fe4d9589")
# Define the query
model = "gpt-4"  # or any other model you are using

system_message = "You are a highly skilled professional bioinformatician with extensive knowledge in bioinformatics analysis and data analysis. You are capable of providing expert insights on complex biological data, including sequence analysis, structural bioinformatics, genomics, and statistical methods for data interpretation. Your expertise also extends to advising on experimental design and data-driven decision making in the field of bioinformatics."


query = "I have uploaded 20 files, with 5 files for each type of RNA sequence. For each RNA type, there are four files representing different experimental conditions and one control file. Build a recurrent neural network-based model to differentiate between experimental and control RNA sequences. For evaluation results, visualize the confusion matrix and precision-recall curves."

# Generate detailed plan
detailed_plan = planner.create_plan(query=query)

# Generate code based on the detailed plan
generated_code = code_generator.generate_code(plan=detailed_plan)

code = extract_python_code(generated_code)

# Generate test cases for the generated code
test_cases = test_generator.generate_tests(
    code=generated_code, plan=detailed_plan)

test_code = extract_python_code(test_cases)
# Save the generated code to a file
with open('generated_code.py', 'w') as code_file:
    code_file.write(code)

# Save the generated test cases to a file
test_filename = 'test_generated_code.py'
with open(test_filename, 'w') as test_file:
    test_file.write(test_code)

#Run the test cases
# def run_tests(test_file: str):
#     # Check if pytest is available; otherwise, use unittest
#     try:
#         result = subprocess.run(['pytest', test_file], capture_output=True, text=True)
#     except FileNotFoundError:
#         result = subprocess.run(['python', '-m', 'unittest', test_file], capture_output=True, text=True)

#     return result

# # Execute the tests
# test_result = run_tests(test_filename)

# # Print test results
# print("Test Results:")
# print(test_result.stdout)

# # Optional: Check for test failures
# if test_result.returncode != 0:
#     print("Some tests failed. Review the test results above for details.")
# else:
#     print("All tests passed successfully.")

# Clean up test files
# os.remove('generated_code.py')
# os.remove(test_filename)

Test failures detected. Regenerating code and tests...
Test failures detected. Regenerating code and tests...
Test failures detected. Regenerating code and tests...
Test failures detected. Regenerating code and tests...
Test failures detected. Regenerating code and tests...
Test failures detected. Regenerating code and tests...
Test failures detected. Regenerating code and tests...
Test failures detected. Regenerating code and tests...
Test failures detected. Regenerating code and tests...


KeyboardInterrupt: 